# Benchmarking cell2location pyro model using softplus/exp for scales

In [1]:
import sys, ast, os
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt 
import matplotlib as mpl
data_type='float32'

#sys.path.insert(1, '/nfs/team205/vk7/sanger_projects/BayraktarLab/cell2location/')
#sys.path.insert(1, '/nfs/team205/vk7/sanger_projects/software/pyro/')

import cell2location_model 

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
import seaborn as sns

`THEANO_FLAGS='force_device=True'` forces the package to use GPU. Pay attention to error messages that might indicate theano failed to initalise the GPU.   
Do not forget to change `device=cuda4` to your available GPU id. Use `device=cuda` / `device=cuda0` if you have just one locally or if you are requesting one GPU via HPC cluster job.

### The purpose of the notebook is to benchmark several versions of the model using mouse brain data.

In [2]:
sc_data_folder = '/nfs/team205/vk7/sanger_projects/cell2location_paper/notebooks/selected_data/mouse_visium_snrna/'
sp_data_folder = '/nfs/team205/vk7/sanger_projects/cell2location_paper/notebooks/selected_results/benchmarking/with_tissue_zones/data/'
results_folder = '/nfs/team205/vk7/sanger_projects/cell2location_paper/notebooks/selected_results/benchmarking/with_tissue_zones/real_mg/pyro/'

## Read datasets and train cell2location

In [3]:
adata_vis = anndata.read(f'{sp_data_folder}synth_adata_real_mg_20210131.h5ad')
adata_vis.uns['spatial'] = {'x': 'y'}
adata_vis = adata_vis[adata_vis.obs['sample'] == 'exper0',:]

adata_snrna_raw = anndata.read(f'{sp_data_folder}training_5705STDY8058280_5705STDY8058281_20210131.h5ad')

/nfs/team283/vk7/software/miniconda3farm5/envs/scvi-env2/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [4]:
import scipy
adata_snrna_raw.X = scipy.sparse.csr_matrix(adata_snrna_raw.X)

In [5]:
adata_vis.X = scipy.sparse.csr_matrix(adata_vis.X)

Add counts matrix as `adata.raw` 

In [6]:
adata_snrna_raw.raw = adata_snrna_raw
adata_vis.raw = adata_vis

In [7]:
# compute average for each cluster
aver = cell2location_model.get_cluster_averages(adata_snrna_raw, 'annotation_1')

# make sure the order of gene matches between aver and x_data
aver = aver.loc[adata_vis.var_names,:]

# generate one-hot encoded matrix telling which obs belong to whic samples
obs2sample_df = pd.get_dummies(adata_vis.obs['sample'])

In [8]:
adata_vis

AnnData object with n_obs × n_vars = 2500 × 12809
    obs: 'cell_count_Inh_3', 'cell_count_Ext_Pir', 'cell_count_Oligo_2', 'cell_count_Astro_HYPO', 'cell_count_Ext_L25', 'cell_count_Astro_THAL_lat', 'cell_count_Ext_L5_1', 'cell_count_Ext_L23', 'cell_count_Micro', 'cell_count_Ext_Thal_1', 'cell_count_Ext_Hpc_CA1', 'cell_count_Inh_Meis2_3', 'cell_count_Inh_4', 'cell_count_OPC_1', 'cell_count_Oligo_1', 'cell_count_Ext_Hpc_CA3', 'cell_count_Astro_CTX', 'cell_count_Astro_THAL_med', 'cell_count_Inh_Vip', 'cell_count_Ext_Amy_2', 'cell_count_Astro_AMY_CTX', 'cell_count_Unk_1', 'cell_count_Inh_Sst', 'cell_count_Inh_Meis2_2', 'cell_count_Ext_Hpc_DG2', 'cell_count_Ext_Hpc_DG1', 'cell_count_Inh_1', 'cell_count_Inh_Meis2_4', 'cell_count_Inh_2', 'cell_count_Nb_1', 'cell_count_Ext_Thal_2', 'cell_count_Ext_Amy_1', 'cell_count_Ext_L6', 'cell_count_OPC_2', 'cell_count_Inh_Meis2_1', 'cell_count_Ext_L6B', 'cell_count_LowQ_2', 'cell_count_Ext_Unk_3', 'cell_count_Ext_L56', 'cell_count_Inh_Lamp5', 'cell_coun

## Model training

In [9]:
results_folder

'/nfs/team205/vk7/sanger_projects/cell2location_paper/notebooks/selected_results/benchmarking/with_tissue_zones/real_mg/pyro/'

In [10]:
#import torch
#torch.set_default_tensor_type(torch.cuda.FloatTensor)

mod = cell2location_model.LocationModelLinearDependentWMultiExperiment(
    n_obs=adata_vis.n_obs,
    n_var=adata_vis.n_vars,
        n_fact=aver.shape[1],
        n_exper=obs2sample_df.shape[1],
        batch_size=None,
        cell_state_mat=aver.values.astype(data_type),
    )

from pyro.distributions import constraints
mod.guide.scale_constraint = constraints.positive
#mod.guide.scale_constraint = constraints.softplus_positive

#mod.to('cuda')
mod._train_full_data(x_data=adata_vis.raw.X.toarray().astype(data_type),
                     obs2sample=obs2sample_df.values.astype(data_type),
                     n_epochs=2000, lr=0.002)

  0%|          | 0/2000 [00:00<?, ?it/s]

/nfs/team283/vk7/software/miniconda3farm5/envs/scvi-env2/lib/python3.7/site-packages/pyro/poutine/subsample_messenger.py:141: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if len(shape) >= -dim and shape[dim] != 1:
/nfs/team283/vk7/software/miniconda3farm5/envs/scvi-env2/lib/python3.7/site-packages/pyro/poutine/subsample_messenger.py:151: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.subsample_size < self.size:
/nfs/team283/vk7/software/miniconda3farm5/envs/scvi-env2/lib/python3.7/site-packages/pyro/poutine/subsample_messenger.py:141: TracerWar

RuntimeError: No grad accumulator for a saved leaf!

In [ ]:
means = mod.guide.median()
means = {k: means[k].cpu().detach().numpy() 
         for k in means.keys()}

In [ ]:
means

In [ ]:
mod.model.m_g_shape

In [ ]:
mod.model.m_g_rate

In [ ]:
import inspect
import test
 
print(inspect.getsource(type(mod.guide)))

In [ ]:
mod.model.cell_number_prior["N_cells_per_location"]

In [ ]:
from torch.distributions import constraints
from torch.distributions.transforms import Transform
from torch.nn.functional import softplus


def softplus_inv(y):
    return y + y.neg().expm1().neg().log()

class SoftplusTransform(Transform):
    r"""
    Transform via the mapping :math:`\text{Softplus}(x) = \log(1 + \exp(x))`.
    """
    domain = constraints.real
    codomain = constraints.positive
    bijective = True
    sign = +1

    def __eq__(self, other):
        return isinstance(other, SoftplusTransform)

    def _call(self, x):
        return softplus(x)

    def _inverse(self, y):
        return softplus_inv(y)

    def log_abs_det_jacobian(self, x, y):
        return -softplus(-x)
    

In [ ]:
#SoftplusTransform()._inverse(torch.tensor(np.array([1])))

In [ ]:
import torch
torch.set_default_tensor_type(torch.cuda.FloatTensor)

mod_s = cell2location_model.LocationModelLinearDependentWMultiExperiment(
    n_obs=adata_vis.n_obs,
    n_var=adata_vis.n_vars,
        n_fact=aver.shape[1],
        n_exper=obs2sample_df.shape[1],
        batch_size=None,
        cell_state_mat=aver.values.astype(data_type),
    )

from pyro.distributions import constraints
mod_s.guide.scale_constraint = constraints.softplus_positive
#mod_s.guide.scale_constraint = softplus_positive

#mod.cuda()
mod_s._train_full_data(x_data=adata_vis.raw.X.toarray().astype(data_type),
                    obs2sample=obs2sample_df.values.astype(data_type),
                    n_epochs=1500, lr=0.002)

In [ ]:
means_softplus = mod_s.guide.median()
means_softplus = {k: means_softplus[k].cpu().detach().numpy() 
                  for k in means_softplus.keys()}
means_softplus

In [ ]:
import torch
torch.set_default_tensor_type(torch.cuda.FloatTensor)

mod_s3 = cell2location_model.LocationModelLinearDependentWMultiExperiment(
    n_obs=adata_vis.n_obs,
    n_var=adata_vis.n_vars,
        n_fact=aver.shape[1],
        n_exper=obs2sample_df.shape[1],
        batch_size=None,
        cell_state_mat=aver.values.astype(data_type),
    )

from pyro.distributions import constraints
from torch.distributions import biject_to, transform_to

@biject_to.register(constraints.positive)
@transform_to.register(constraints.positive)
def _transform_to_positive(constraint):
    return SoftplusTransform()

mod_s3.guide.scale_constraint = constraints.positive

#mod.cuda()
mod_s3._train_full_data(x_data=adata_vis.raw.X.toarray().astype(data_type),
                    obs2sample=obs2sample_df.values.astype(data_type),
                    n_epochs=1500, lr=0.002)

### Compare ELBO as training progresses

In [ ]:
plt.plot(range(2000, len(mod.hist)), np.array(mod.hist)[2000:]);
plt.plot(range(2000, len(mod.hist)), np.array(mod_s.hist)[2000:]);
plt.plot(range(2000, len(mod.hist)), np.array(mod_s3.hist)[2000:]);
plt.legend(labels=['exp', 'softplus scales', 'all_softplus']);
plt.xlim(0, len(mod.hist));

### Evaluate accuracy using $R^2$

In [ ]:
means = mod.guide.median()
means = {k: means[k].cpu().detach().numpy() 
         for k in means.keys()}
means_softplus = mod_s.guide.median()
means_softplus = {k: means_softplus[k].cpu().detach().numpy() 
                  for k in means_softplus.keys()}
means_softplus_all = mod_s3.guide.median()
means_softplus_all = {k: means_softplus_all[k].cpu().detach().numpy() 
                  for k in means_softplus_all.keys()}

In [ ]:
from re import sub
cell_count = adata_vis.obs.loc[:, ['cell_abundances_' in i for i in adata_vis.obs.columns]]
cell_count.columns =  [sub('cell_abundances_', '', i) for i in cell_count.columns]
cell_count_columns = cell_count.columns

infer_cell_count = pd.DataFrame(means['w_sf'], index=adata_vis.obs_names, 
                                columns=aver.columns)
infer_cell_count = infer_cell_count[cell_count.columns]

infer_cell_count_softplus = pd.DataFrame(means_softplus['w_sf'], index=adata_vis.obs_names, 
                                columns=aver.columns)
infer_cell_count_softplus = infer_cell_count_softplus[cell_count.columns]

infer_cell_count_softplus_all = pd.DataFrame(means_softplus_all['w_sf'], index=adata_vis.obs_names, 
                                columns=aver.columns)
infer_cell_count_softplus_all = infer_cell_count_softplus_all[cell_count.columns]

In [ ]:
infer_cell_count.iloc[0:5,0:5], infer_cell_count_softplus.iloc[0:5,0:5], infer_cell_count_softplus_all.iloc[0:5,0:5]

In [ ]:
rcParams['figure.figsize'] = 4, 4
rcParams["axes.facecolor"] = "white"

plt.hist2d(cell_count.values.flatten(), 
           infer_cell_count.values.flatten(),# / np.mean(adata_vis_res.var['gene_level'].values),
           bins=[50, 50], norm=mpl.colors.LogNorm());
plt.xlabel('Simulated cell abundance');
plt.ylabel('Estimated cell abundance');
plt.title(r'exp, $R^2$: ' \
          + str(np.round(np.corrcoef(cell_count.values.flatten(), 
           infer_cell_count.values.flatten()), 3)[0,1]));
#plt.gca().set_aspect('equal', adjustable='box')
plt.tight_layout()
#plt.savefig(fig_path + '/Cell_density_cor.pdf')

In [ ]:
rcParams['figure.figsize'] = 4, 4
rcParams["axes.facecolor"] = "white"

plt.hist2d(cell_count.values.flatten(), 
           infer_cell_count_softplus.values.flatten(),# / np.mean(adata_vis_res.var['gene_level'].values),
           bins=[50, 50], norm=mpl.colors.LogNorm());
plt.xlabel('Simulated cell abundance');
plt.ylabel('Estimated cell abundance');
plt.title(r'softplus, $R^2$: ' \
          + str(np.round(np.corrcoef(cell_count.values.flatten(), 
           infer_cell_count_softplus.values.flatten()), 3)[0,1]));
#plt.gca().set_aspect('equal', adjustable='box')
plt.tight_layout()
#plt.savefig(fig_path + '/Cell_density_cor.pdf')

In [ ]:
rcParams['figure.figsize'] = 4, 4
rcParams["axes.facecolor"] = "white"

plt.hist2d(cell_count.values.flatten(), 
           infer_cell_count_softplus_all.values.flatten(),# / np.mean(adata_vis_res.var['gene_level'].values),
           bins=[50, 50], norm=mpl.colors.LogNorm());
plt.xlabel('Simulated cell abundance');
plt.ylabel('Estimated cell abundance');
plt.title(r'softplus all, $R^2$: ' \
          + str(np.round(np.corrcoef(cell_count.values.flatten(), 
           infer_cell_count_softplus_all.values.flatten()), 3)[0,1]));
#plt.gca().set_aspect('equal', adjustable='box')
plt.tight_layout()
#plt.savefig(fig_path + '/Cell_density_cor.pdf')